In [ ]:
# LSTM Series

# Matlab initialization strategy
# Train and Dev different loss functions
# Adam Optimizer
#

In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import math

import scipy.io as sio
import numpy as np
import h5py
import tensorflow as tf
import sys

import time

In [ ]:
'''
Read parameters from .mat files
save in Opt object
'''
################################################################

In [8]:
Data_VERSION = "_e10v1"
Code_VERSION = "_e11v1"

In [9]:
DNN_DATA_FILE = "./dnn_models/DNN_datas"+ Data_VERSION+".mat"
# DNN_DATA_FILE = "./dnn_models/BR2_DNN_datas.mat"

DNN_MODEL_FILE = "./dnn_models/DNN_params"+ Data_VERSION+".mat"
# DNN_MODEL_FILE = "./dnn_models/BR2_DNN_params.mat"

DNN_NET_FILE = "./dnn_models/DNN_net"+ Code_VERSION+".mat"

# ModelFN_FILE = "./dnn_models/dnncirm.noiseSSN_05.mat"

In [10]:
class Opts:
    opts_dict = dict()

    def __init__(self, FILE, FILE_DATA):
        with h5py.File(FILE, 'r') as f:
            key_list = list(f.keys())
            print('Opt key:',key_list)

            for k, v in f['opts'].items():

                print('key:', k)

                if k == 'ARMA_order':
                    self.ARMA_order = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.ARMA_order
                elif k == 'ada_grad_eps':
                    self.ada_grad_eps = np.array(v)[0][0]
                    self.opts_dict[k] = self.ada_grad_eps
                elif k == 'ada_sgd_scale':
                    self.ada_sgd_scale = np.array(v)[0][0]
                    self.opts_dict[k] = self.ada_sgd_scale
                elif k == 'change_momentum_point':
                    self.change_momentum_point = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.change_momentum_point
                elif k == 'cost_function':
                    self.cost_function = ""
                    for c in np.array(v):
                        self.cost_function += chr(c[0])

                    self.opts_dict[k] = self.cost_function

                elif k == 'cv_interval':
                    self.cv_interval = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.cv_interval
                elif k == 'dim_input':
                    self.dim_input = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.dim_input
                elif k == 'dim_output':
                    self.dim_output = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.dim_output
                elif k == 'drop_ratio':
                    self.drop_ratio = np.array(v)[0][0]
                    self.opts_dict[k] = self.drop_ratio
                elif k == 'eval_on_gpu':
                    self.eval_on_gpu = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.eval_on_gpu
                elif k == 'final_momentum':
                    self.final_momentum = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.final_momentum
                elif k == 'hid_struct':
                    self.hid_struct = np.array(v)
                    self.opts_dict[k] = self.hid_struct
                elif k == 'initial_momentum':
                    self.initial_momentum = np.array(v)[0][0]
                    self.opts_dict[k] = self.initial_momentum
                elif k == 'isDropout':
                    self.isDropout = 0
                    self.opts_dict[k] = self.isDropout
                elif k == 'isDropoutInput':
                    self.isDropoutInput = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isDropoutInput
                elif k == 'isGPU':
                    self.isGPU = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isGPU
                elif k == 'isNormalize':
                    self.isNormalize = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isNormalize
                elif k == 'isPretrain':
                    self.isPretrain = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isPretrain
                elif k == 'learner':
                    self.learner = ""
                    for c in np.array(v):
                        self.learner += chr(c[0])

                    self.opts_dict[k] = self.learner

                elif k == 'net_struct':
                    self.net_struct = np.array(v)
                    for n_s in np.array(v):
                        print('Opts Net Stuct:',n_s[0])

                    self.opts_dict[k] = self.net_struct
                elif k == 'rbm_batch_size':
                    self.rbm_batch_size = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.rbm_batch_size
                elif k == 'rbm_learn_rate_binary':
                    self.rbm_learn_rate_binary = np.array(v)
                    # self.opts_dict[k] = self.rbm_learn_rate_binary
                elif k == 'rbm_learn_rate_real':
                    self.rbm_learn_rate_real = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.rbm_learn_rate_real
                elif k == 'rbm_max_epoch':
                    self.rbm_max_epoch = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.rbm_max_epoch
                elif k == 'save_on_fly':
                    self.save_on_fly = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.save_on_fly
                elif k == 'sgd_batch_size':
                    self.sgd_batch_size = int(np.array(v)[0][0])
                    self.sgd_batch_size = 1024//16
                    # self.opts_dict[k] = self.sgd_batch_size
                elif k == 'sgd_learn_rate':
                    self.sgd_learn_rate = np.array(v)
                    # self.opts_dict[k] = self.sgd_learn_rate
                elif k == 'sgd_max_epoch':
                    self.sgd_max_epoch = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.sgd_max_epoch
                elif k == 'split_tanh1_c1':
                    self.split_tanh1_c1 = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.split_tanh1_c1
                elif k == 'split_tanh1_c2':
                    self.split_tanh1_c2 = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.split_tanh1_c2
                elif k == 'unit_type_hidden':
                    self.unit_type_hidden = ""
                    for c in np.array(v):
                        self.unit_type_hidden += chr(c[0])

                elif k == 'unit_type_output':
                    self.unit_type_output = ""
                    for c in np.array(v):
                        self.unit_type_output += chr(c[0])

        with h5py.File(FILE_DATA, 'r') as f:
            print('Opts h5py keys:', list(f.keys()))
            for k, v in f.items():

                if k == 'trData':
                    print("trData.shape: ", v.shape)
#                     self.trData = np.transpose(np.array(v))
#                     print("trData-> mean:", np.mean(self.trData),", var:",np.var(self.trData), ", std:",np.std(self.trData))
                elif k == 'trLabel_i':
                    print("trLabel_i.shape: ", v.shape)
#                     self.trLabel_i = np.transpose(np.array(v))
#                     print("trLabel_i-> mean:", np.mean(self.trLabel_i), ", var:", np.var(self.trLabel_i), ", std:",
#                           np.std(self.trLabel_i))
                elif k == 'trLabel_r':
                    print("trLabel_r.shape: ", v.shape)
#                     self.trLabel_r = np.transpose(np.array(v))
#                     print("trLabel_r-> mean:", np.mean(self.trLabel_r), ", var:", np.var(self.trLabel_r), ", std:",
#                           np.std(self.trLabel_r))
                elif k == 'cvData':
                     print("cvData.shape: ", v.shape)
#                     self.cvData = np.transpose(np.array(v))
#                     print("cvData-> mean:", np.mean(self.cvData), ", var:", np.var(self.cvData), ", std:",
#                           np.std(self.cvData))
                elif k == 'cvLabel_i':
                    print("cvLabel_i.shape: ", v.shape)
#                     self.cvLabel_i = np.transpose(np.array(v))
#                     print("cvLabel_i-> mean:", np.mean(self.cvLabel_i), ", var:", np.var(self.cvLabel_i), ", std:",
#                           np.std(self.cvLabel_i))
                elif k == 'cvLabel_r':
                    print("cvLabel_r.shape: ", v.shape)
#                     self.cvLabel_r = np.transpose(np.array(v))
#                     print("cvLabel_r-> mean:", np.mean(self.cvLabel_r), ", var:", np.var(self.cvLabel_r), ", std:",
#                           np.std(self.cvLabel_r))

#             self.trLabel = np.concatenate((self.trLabel_r, self.trLabel_i), axis=1)
#             self.cvLabel = np.concatenate((self.cvLabel_r, self.cvLabel_i), axis=1)


    def ready_batchID(self, total_num_samples, batch_size):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        batchID = []
        num_batch = math.ceil(total_num_samples/batch_size)

        for b in range( int(num_batch) ):
            s = b*batch_size
            e = (b+1)*batch_size -1

            if e >= total_num_samples:
                e = total_num_samples - 1

            batchID.append((s,e))

        return np.array(batchID,ndmin=2)


    def suffle_data(self, total_num_samples):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        return  np.random.permutation(total_num_samples)


    def next_batch(self, total_num_samples, batch_size, isTrainCycle=True):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        batchID = self.ready_batchID(total_num_samples, batch_size)
        seq = self.suffle_data(total_num_samples)

        for batch in range(batchID.shape[0]):
            if isTrainCycle:
                x = opts.trData[ seq[batchID[batch][0]:batchID[batch][1] ] ]
                y = opts.trLabel[ seq[batchID[batch][0]:batchID[batch][1] ] ]

            else:
                x = opts.cvData[seq[batchID[batch][0]:batchID[batch][1]]]
                y = opts.cvLabel[seq[batchID[batch][0]:batchID[batch][1]]]

            # print('Next Batch', x.shape, y.shape)
            yield [x, y]


    def next_batch2(self, batch_size, isTrainCycle=True):
        if isTrainCycle:
            # selected_indics = np.random.choice(10, size=batch_size, replace=False)
            selected_indics = np.random.randint(195192, size=batch_size)
        else:
            # selected_indics = np.random.randint(7, size= batch_size)
            selected_indics = np.random.randint(44961, size=batch_size)

        # print ("selected_indics: ",selected_indics)
        if isTrainCycle:
            x = opts.trData[selected_indics]
            y = np.concatenate((opts.trLabel_r, opts.trLabel_i), axis=1)[selected_indics]

        else:
            x = opts.cvData[selected_indics]
            y = np.concatenate((opts.cvLabel_r, opts.cvLabel_i), axis=1)[selected_indics]

        # print('Next Batch', x.shape, y.shape)
        return [x, y]

In [11]:
opts = Opts(DNN_MODEL_FILE, DNN_DATA_FILE)

Opt key: ['#refs#', 'opts']
key: ARMA_order
key: ada_grad_eps
key: ada_sgd_scale
key: change_momentum_point
key: cost_function
key: cv_interval
key: dim_input
key: dim_output
key: drop_ratio
key: eval_on_gpu
key: final_momentum
key: hid_struct
key: initial_momentum
key: isDropout
key: isDropoutInput
key: isGPU
key: isNormalize
key: isPretrain
key: learner
key: net_struct
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
key: rbm_batch_size
key: rbm_learn_rate_binary
key: rbm_learn_rate_real
key: rbm_max_epoch
key: save_on_fly
key: sgd_batch_size
key: sgd_learn_rate
key: sgd_max_epoch
key: split_tanh1_c1
key: split_tanh1_c2
key: tr_mu
key: tr_std
key: unit_type_hidden
key: unit_type_output
Opts h5py keys: ['cvData', 'cvLabel_i', 'cvLabel_r', 'trData', 'trLabel_i', 'trLabel_r']
cvData.shape:  (1230, 449610)
cvLabel_i.shape:  (963, 449610)
c